In [1]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    #this changing of this value will be captured by traitlets
    frame = traitlets.Any()
    
    def __init__(self):
        print("attempting to initialise camera...")
        super(Camera, self).__init__()
        
        #create the sensor pipeline and config.
        self.video_capture = cv2.VideoCapture(-1)
        
        #set resolution for the color camera
        self.width = 640
        self.height = 480
        self.fps = 30

        #flag to control the thread
        self.thread_runnning_flag = False 

        self.ready = True
        
        #start capture the first color image
        frame_success, frame = self.video_capture.read()
        self.frame = frame
        
    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frame_success, frame = self.video_capture.read()
            self.frame = frame
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data
if(camera.ready):
    print("CAMERA READY")

attempting to initialise camera...
CAMERA READY


In [ ]:
from mtcnn import MTCNN
from SixDRepNet import SixDRepNet

face_detector = MTCNN()
pose_detector = SixDRepNet()

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import time

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

def resize_with_padding(image,target_size):
    old_size= image.shape[:2]
    print(old_size)
    ratio = float(target_size) / max(old_size)
    new_dim = (int(old_size[1] * ratio),int(old_size[0] * ratio))
    print(new_dim)

    image = cv2.resize(image, new_dim)

    return image

image_widget = widgets.Image(format='jpeg', width=512, height=512)
label_widget = widgets.Label(value="default")
display(image_widget,label_widget)

def processing(change):
    frame = change["new"]

    face_start = time.time()
    detection = face_detector.detect_faces(frame)
    face_duration = time.time() - face_start
    for face in detection:
        bbox = face["box"]
        # create raw bounding box in blue
        frame = cv2.rectangle(frame,
        (bbox[0],bbox[1]),
        (bbox[0] + bbox[2],bbox[1] + bbox[3]),
        (255,127,0),
        2)

        # create square centered bounding box in orange

        box_size = max(bbox[2:4])
        top_left = (int(bbox[0] + (bbox[2]/2) - (box_size/2)),int(bbox[1] + (bbox[3]/2) - (box_size/2)))
        
        frame = cv2.rectangle(frame,
        top_left,
        (top_left[0] + box_size,top_left[1] + box_size),
        (0,127,255),
        2)

        cropped = frame[top_left[1]:top_left[1]+box_size,top_left[0]:top_left[0]+box_size]

        pose_start = time.time()
        pitch, yaw, roll = pose_detector.predict(cropped)
        pose_duration = time.time() - pose_start()
        pose_detector.draw_axis(frame, yaw, pitch, roll,int(top_left[0] + box_size/2),int(top_left[1] + box_size/2))

        total_duration = time.time()-face_start

        image_widget.value = bgr8_to_jpeg(frame)

        '''
        label_widget.value = """
        face: {:.2f}s
        pose: {:.2f}s
        total: {:.2f}s
        """.format(face_duration,pose_duration,total_duration)
        '''

camera.observe(processing,names="frame")